# MNIST 세이지 메이커 기본 훈련 및 DDP 훈련

# 1. 기본 세팅

In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [3]:
! df -h
# ! docker container prune -f
# ! docker image prune -f --all
# ! rm -rf /tmp/tmp*
# ! df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        241G   80K  241G   1% /dev
tmpfs           241G     0  241G   0% /dev/shm
/dev/xvda1      109G   95G   14G  88% /
/dev/xvdf       984G   24G  911G   3% /home/ec2-user/SageMaker


# 2. 기본 훈련

In [4]:
from sagemaker.tensorflow import TensorFlow


instance_type = "local_gpu"

estimator = TensorFlow(
    entry_point="mnist_train.py",    
    source_dir="src",
    role=role,
    py_version="py37",
    framework_version="2.4.1",
    # For training with multinode distributed training, set this count. Example: 2
    instance_count=1,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type=instance_type,

)

estimator.fit(wait=False)

Creating 93w3xotbzp-algo-1-yejsf ... 
Creating 93w3xotbzp-algo-1-yejsf ... done
Attaching to 93w3xotbzp-algo-1-yejsf
93w3xotbzp-algo-1-yejsf | 2021-10-04 05:10:47.706396: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
93w3xotbzp-algo-1-yejsf | 2021-10-04 05:10:47.706591: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
93w3xotbzp-algo-1-yejsf | 2021-10-04 05:10:47.711064: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
93w3xotbzp-algo-1-yejsf | 2021-10-04 05:10:47.749773: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
93w3xotbzp-algo-1-yejsf | 2021-10-04 05:10:49,595 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
93w3xotbzp-algo

# 3. DDP Local Mode

In [5]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    base_job_name="tensorflow2-smdataparallel-mnist",
    source_dir="src",
    entry_point="mnist_train_DDP.py",    
    role=role,
    py_version="py37",
    framework_version="2.4.1",
    instance_count=1,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type= instance_type,
#    sagemaker_session=sagemaker_session,
    # Training using SMDataParallel Distributed Training Framework
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
)

In [6]:
estimator.fit()

Creating 7rrh9djrip-algo-1-gy43h ... 
Creating 7rrh9djrip-algo-1-gy43h ... done
Attaching to 7rrh9djrip-algo-1-gy43h
7rrh9djrip-algo-1-gy43h | 2021-10-04 05:12:12.495853: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
7rrh9djrip-algo-1-gy43h | 2021-10-04 05:12:12.496041: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
7rrh9djrip-algo-1-gy43h | 2021-10-04 05:12:12.500470: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
7rrh9djrip-algo-1-gy43h | 2021-10-04 05:12:12.538159: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
7rrh9djrip-algo-1-gy43h | 2021-10-04 05:12:14,282 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
7rrh9djrip-algo

# 4. DDP SM Hostmode

In [7]:
from sagemaker.tensorflow import TensorFlow

instance_type = 'ml.p3.16xlarge'

estimator = TensorFlow(
    base_job_name="tensorflow2-smdataparallel-mnist",
    source_dir="src",
    entry_point="mnist_train_DDP.py",    
    role=role,
    py_version="py37",
    framework_version="2.4.1",
    instance_count=2,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type= instance_type,
#    sagemaker_session=sagemaker_session,
    # Training using SMDataParallel Distributed Training Framework
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
)

estimator.fit(wait=False)

In [8]:
estimator.logs()

2021-10-02 08:01:24 Starting - Starting the training job...
2021-10-02 08:01:49 Starting - Launching requested ML instancesProfilerReport-1633161684: InProgress
............
2021-10-02 08:03:50 Starting - Preparing the instances for training.........
2021-10-02 08:05:18 Downloading - Downloading input data
2021-10-02 08:05:18 Training - Downloading the training image...............
2021-10-02 08:07:51 Training - Training image download completed. Training in progress..2021-10-02 08:07:49.844216: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-10-02 08:07:49.848628: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-10-02 08:07:49.934099: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-02 08:07:50.032883: W tensorf